<a href="https://colab.research.google.com/github/ykim71/georgia_analysis/blob/main/(3)_Georgia_topic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# need access to the folder(s) below:

#%cd /content/drive/MyDrive/Georgia/


/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Locations Analysis


# Topic analysis

## Prep for topic analysis

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries

import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')


In [ ]:
# lemmatization if needed
import spacy
import re

nlp = spacy.load("en_core_web_sm", disable=['ner', 'parser']) # disabling Named Entity Recognition for speed


/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
import re

def get_lemmas_num(text):

    regex = re.compile('[^a-zA-Z0-9]')

    lemmas = []
    
    doc = nlp(text)

    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON') and (token.text != 'NA'):
            lemmas.append(regex.sub('', token.lemma_))
    
    return ' '.join(lemmas)

In [ ]:
df['article_lemma_num'] = df['article_final'].str.lower().astype(str).apply(get_lemmas_num)


In [ ]:
#df_source_merge_final.to_pickle('Georgia_data_v2_translation_ner_lem.pkl') # commented this line to prevent from saving it

## word2vec training

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries

import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')


In [ ]:
import pandas as pd
# To extract n-gram from text
from gensim.models.phrases import Phrases, Phraser
# To train word2vec
from gensim.models import Word2Vec
# To load pre trained word2vec
from gensim.models import KeyedVectors


In [ ]:
df.columns

Index(['source', 'en_source', 'website', 'article_url', 'article_title',
       'article_author', 'date_published', 'article_final', 'ner_places',
       'ner_orgs', 'article_lemma_num'],
      dtype='object')

In [ ]:
# As Phrases() takes list of list as it's input
# Converting dataframe to list of list

## all lower cases

sentence_stream = [doc.split(" ") for doc in df['article_lemma_num']]
bigram = Phrases(sentence_stream, min_count= 10, delimiter=b' ') # v2 threshold as default
bigram_phraser = Phraser(bigram)

# bigram = Phrases(sentence_stream, min_count=10, threshold= 10, delimiter=b' ') # higher threshold fewer phrases.
# trigram = Phrases(bigram[sentence_stream], threshold= 100, delimiter=b' ') # include bi- and tri-grams


# Extract bigram for gensim word2vec
bigram_token = []

for sent in sentence_stream:
    bigram_token.append(bigram_phraser[sent])

article_model_bigram = Word2Vec(bigram_token, min_count=10, size= 40, workers=5, window = 5, sg = 0)
article_model_bigram.save("Georgia_language_lemma_num10_v2") # v2 : with updated dataset


## topic words identification

### separate analysis for words with asterisk

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries


In [ ]:
topics = pd.read_excel("topics.xlsx")

topics['word_lemma'] = topics['word'].str.lower().astype(str).apply(get_lemmas_num)


In [ ]:
import numpy as np

star_phrases = topics['word'].str.contains('\*{1}', case=False, regex=True)

topics['word_star'] = np.where(star_phrases, 1, 0)


In [ ]:
topics_st = topics[['topic', 'word_lemma', 'word_star']].drop_duplicates()
topics_st.sample(3)

,topic,word_lemma,word_star
376,Education,kindergarten,0
411,Crime,assault,0
250,Climate change,endanger specie,0


In [ ]:
topics_st_asterisk = topics_st[topics_st['word_star']==1]

In [ ]:
topics_st_asterisk_list = topics_st_asterisk['word_lemma'].values.tolist()

In [ ]:
words = pd.DataFrame(list(article_model_bigram.wv.vocab.keys()))
words.rename(columns={0:'n_grams'}, inplace=True)


In [ ]:
words['word_lemma'] = words['n_grams'].astype(str).apply(lambda x: [i for i in topics_st_asterisk_list if i in x])


In [ ]:
words_filter = words[ words['word_lemma'].str.len() > 0 ]

In [ ]:
words_filter_unnest = words_filter.explode('word_lemma')


In [ ]:
words_filter_unnest = pd.merge(words_filter_unnest, topics_st_asterisk, on="word_lemma", how='left')

In [ ]:
words_filter_unnest.to_excel('additional_topic_words_v2.xlsx')

In [ ]:
additional_topic_words_v2 = pd.read_excel('additional_topic_words_v2.xlsx')
additional_topic_words_v1 = pd.read_excel('additional_topic_words.xlsx')

In [ ]:
additional_topic_words_v1['version'] = 'v1'
additional_topic_words_v2['version'] = 'v2'

add_topic_words_merge = pd.merge( additional_topic_words_v1, additional_topic_words_v2, how="outer", on=["n_grams", "topic"])

In [ ]:
add_topic_words_merge = add_topic_words_merge[['n_grams', 'topic', 'FINAL', 'version_x', 'version_y']]

In [ ]:
add_topic_words_merge.to_excel('Topic_words_review1_v1_v2_diff.xlsx')


In [ ]:
#add_topic_words_merge_NA = add_topic_words_merge[ (add_topic_words_merge['version_x'].isnull()) | (add_topic_words_merge['version_y'].isnull()) ]


### final list prep

In [ ]:
add_words = pd.read_excel("/content/drive/MyDrive/Georgia/Topic Dictionaries/Topic_words_review1_v1_v2_diff.xlsx")

In [ ]:
topics_R1 = topics_st[topics_st['word_star'] == 0]
topics_R1 = topics_R1[['topic', 'word_lemma']]

In [ ]:
topics_add = add_words[add_words['FINAL2'] == 1]
topics_add = topics_add[['topic', 'n_grams']]
topics_add.rename(columns={'n_grams': 'word_lemma'}, inplace=True)


In [ ]:
topic_final = pd.concat([topics_R1, topics_add])

In [ ]:
topic_final = topic_final.drop_duplicates()

In [ ]:
topic_final

,topic,word_lemma
0,Inflation,cost live
1,Inflation,inflation
3,Inflation,afford
4,Inflation,affordability
5,Inflation,affordable
...,...,...
2848,Racial inequality,minority
2858,Racial inequality,nondiscrimination
2882,Racial inequality,stigma
2890,Racial inequality,stigmatize


## topic words similarity

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries

import gensim

article_model_bigram = gensim.models.Word2Vec.load("Georgia_language_lemma_num10_v2")


In [ ]:
topic_dictionary = list(set(topic_final.word_lemma.tolist()))

In [ ]:
## update this code to create a dataset with topic words; do it again among the lsd words

word_list_scores = []

for topic_word in topic_dictionary:
  try:
    word_list_scores.append( (topic_word, article_model_bigram.wv.most_similar(topic_word, topn=10)) )
  except KeyError:
    word_list_scores.append( (topic_word, 'NA') )

In [ ]:
df_test = pd.DataFrame ( word_list_scores )


In [ ]:
df_test.rename(columns={0: 'word_lemma', 1: 'similar_words'}, inplace=True)

df_text_spread = df_test.explode('similar_words')

In [ ]:
# df_text_spread_NA = df_text_spread[df_text_spread['similar_words']=='NA']
# df_text_spread_NA.to_excel('FINA_topic_words_not_include_review.xlsx')

In [ ]:
df_text_spread = df_text_spread[df_text_spread['similar_words']!='NA']
df_text_spread['similar_word'], df_text_spread['similarity'] = df_text_spread.similar_words.str[0], df_text_spread.similar_words.str[1]
df_text_spread = df_text_spread.drop('similar_words', axis=1)

<ipython-input-80-b3f5d5179380>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text_spread['similar_word'], df_text_spread['similarity'] = df_text_spread.similar_words.str[0], df_text_spread.similar_words.str[1]


In [ ]:
df_text_spread_merge = pd.merge(df_text_spread, topic_final, how='left', on='word_lemma')

In [ ]:
# those with higher than 0.7 similarity

df_test_sim07 = df_text_spread_merge[ df_text_spread_merge['similarity'] > 0.7  ]

In [ ]:
# 4681 previous
# 4391 current

In [ ]:
df_test_sim07 # 4681

,word_lemma,similar_word,similarity,topic
10,recycle,compost,0.844696,Climate change
11,recycle,reusable,0.805904,Climate change
12,recycle,container,0.798560,Climate change
13,recycle,reuse,0.794733,Climate change
14,recycle,recyclable,0.794667,Climate change
...,...,...,...,...
5105,salary,retention bonus,0.715568,Economy / Jobs
5106,salary,wage,0.714138,Economy / Jobs
5107,salary,pension,0.713663,Economy / Jobs
5108,salary,earning,0.710439,Economy / Jobs


In [ ]:
df_test_sim07_black = df_test_sim07[df_test_sim07['word_lemma']=='black']
df_test_sim07_black

,word_lemma,similar_word,similarity,topic
4420,black,white,0.844675,Racial inequality
4421,black,color,0.715145,Racial inequality


In [ ]:
df_test_sim07_nondup = df_test_sim07[['similar_word', 'topic']].drop_duplicates()

In [ ]:
df_test_sim07_nondup = df_test_sim07.drop_duplicates(subset=['similar_word', 'topic'])

In [ ]:
df_test_sim07_nondup # 3289 non-duplicates --> 3145

,word_lemma,similar_word,similarity,topic
10,recycle,compost,0.844696,Climate change
11,recycle,reusable,0.805904,Climate change
12,recycle,container,0.798560,Climate change
13,recycle,reuse,0.794733,Climate change
14,recycle,recyclable,0.794667,Climate change
...,...,...,...,...
5095,vaccinated,prevalence,0.778021,COVID-19
5096,vaccinated,severe illness,0.774695,COVID-19
5097,vaccinated,virus cause,0.773555,COVID-19
5098,vaccinated,syndrome mutation,0.771601,COVID-19


In [ ]:
# exclude those from the topic dictionary
df_test_sim07_final = df_test_sim07_nondup[~df_test_sim07_nondup['similar_word'].isin(topic_dictionary)]


In [ ]:
df_test_sim07_final.to_excel('df_test_sim07_top10_no_duplicates2_v2.xlsx')

In [ ]:
df_test_sim07_top10_v1 = pd.read_excel('df_test_sim07_top10_no_duplicates2.xlsx')
df_test_sim07_top10_v2 = pd.read_excel('df_test_sim07_top10_no_duplicates2_v2.xlsx')



In [ ]:
df_test_sim07_top10_v1 = df_test_sim07_top10_v1[['similar_word', 'topic', 'FINAL']]

In [ ]:
df_test_sim07_top10_v2 = df_test_sim07_top10_v2[['similar_word', 'topic']]

In [ ]:
df_test_sim07_top10_v2['version'] = 'v2'

<ipython-input-145-1b2dc470a668>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_sim07_top10_v2['version'] = 'v2'


In [ ]:
df_sim07_top10_merge = pd.merge( df_test_sim07_top10_v1, df_test_sim07_top10_v2, how="outer", on=["similar_word", "topic"])

In [ ]:
df_sim07_top10_merge[df_sim07_top10_merge['version'].isnull()]

,similar_word,topic,FINAL,version
2,exception,Abortion,0.0,NaN
4,conservative,Abortion,0.0,NaN
5,expand electorate,Abortion,0.0,NaN
6,fellow democrat,Abortion,0.0,NaN
7,gop candidate,Abortion,0.0,NaN
...,...,...,...,...
2886,woke,Racial inequality,0.0,NaN
2891,court decision,Supreme Court,0.0,NaN
2892,court ruling,Supreme Court,0.0,NaN
2893,dobb,Supreme Court,0.0,NaN


In [ ]:
df_sim07_top10_merge[df_sim07_top10_merge['version']=='v2']

,similar_word,topic,FINAL,version
0,abortion ban,Abortion,0.0,v2
1,ban abortion,Abortion,0.0,v2
3,oppose abortion,Abortion,0.0,v2
12,choose,Abortion,0.0,v2
13,prefer,Abortion,0.0,v2
...,...,...,...,...
4545,prevalence,COVID-19,NaN,v2
4546,severe illness,COVID-19,NaN,v2
4547,virus cause,COVID-19,NaN,v2
4548,syndrome mutation,COVID-19,NaN,v2


In [ ]:
df_sim07_top10_merge.to_excel('df_sim07_top10_merge_v1_v2_diff.xlsx')

In [ ]:
topic_final_r1 = topic_final.copy()
topic_final_r1['topic_final'] = 1

In [ ]:
topic_final_r1.rename(columns={'word_lemma': 'similar_word'}, inplace=True)


In [ ]:
df_sim07_top10_merge_all = pd.merge(df_sim07_top10_merge, topic_final_r1, how="outer", on=["similar_word", "topic"])

In [ ]:
df_sim07_top10_merge_all['review_done'] = np.where(  (df_sim07_top10_merge_all['FINAL']== 0) & (df_sim07_top10_merge_all['version']== "v2"), 1, 0  )


In [ ]:
df_sim07_top10_merge_all.to_excel('df_sim07_top10_merge_v1_v2_diff.xlsx')

## final topic word list

In [ ]:
import pandas as pd
topic_R2 = pd.read_excel('df_test_sim07_top10_no_duplicates2.xlsx')

In [ ]:
topic_R2 = topic_R2[topic_R2['FINAL'] == 1]
topic_R2 = topic_R2[['topic', 'similar_word']]
topic_R2.rename(columns={'similar_word': 'word_lemma'}, inplace=True)


In [ ]:
topic_R2_add = pd.read_excel('additional_words_manual.xlsx') 
topic_R2_add.rename(columns={'similar_word': 'word_lemma'}, inplace=True)


In [ ]:
topic_final

,topic,word_lemma
0,Inflation,cost live
1,Inflation,inflation
3,Inflation,afford
4,Inflation,affordability
5,Inflation,affordable
...,...,...
2082,Racial inequality,marginalize
2085,Racial inequality,marginalized
2092,Racial inequality,minority
2107,Racial inequality,stigma


In [ ]:
topic_final[topic_final['word_lemma']=='undocumente']

,topic,word_lemma
122,Immigration / border security,undocumente


In [ ]:
topic_final2 = pd.concat([topic_final, topic_R2, topic_R2_add])

In [ ]:
topic_final2

,topic,word_lemma
0,Inflation,cost live
1,Inflation,inflation
3,Inflation,afford
4,Inflation,affordability
5,Inflation,affordable
...,...,...
0,Abortion,prochoice
1,Abortion,antichoice
2,Abortion,dobbs
3,Crime,gunfire


In [ ]:
len(topic_final2)

940

In [ ]:
topic_final2 = topic_final2.drop_duplicates().sort_values(['topic','word_lemma']).reset_index()

In [ ]:
topic_final2 = topic_final2[['topic','word_lemma']]

In [ ]:
# delete depression AND lay from Economy / Jobs	
# delete traffic from Immigration / border security AND Crime

In [ ]:
topic_final2[topic_final2['word_lemma'] == 'undocumented'] # 467


,topic,word_lemma
821,Immigration / border security,undocumented


In [ ]:
topic_final2[topic_final2['word_lemma'] == 'depression'] # 418
topic_final2[topic_final2['word_lemma'] == 'lay'] # 467
topic_final2[topic_final2['word_lemma'] == 'traffic'] # 371, 818
topic_final2[topic_final2['word_lemma'] == 'undocumente'] # 820

,topic,word_lemma
820,Immigration / border security,undocumente


In [ ]:
topic_final2[topic_final2['word_lemma'] == 'cap trade '] # 94

,topic,word_lemma
94,Climate change,cap trade


In [ ]:
topic_final2.drop([94, 418,467, 371, 818, 820], axis=0, inplace=True)


In [ ]:
topic_final2.to_excel('FINAL_topic_words_R2_review.xlsx')

## mturk review for v2

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries

import pandas as pd

review_v2 = pd.read_excel('/content/drive/MyDrive/Georgia/Topic Dictionaries/FINAL_sim07_top10_merge.xlsx')

/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Topic Dictionaries


In [ ]:
review_v2_review = review_v2[ (review_v2['FINAL_ADD']==1)]

In [ ]:
review_v2_review_st = review_v2_review[['similar_word', 'topic']]

In [ ]:
len(review_v2_review_st)

317

## find example sentences

In [ ]:
import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')


In [ ]:
len(df[df['article_lemma_num'].isnull()])

0

### round 1

In [ ]:
topic_final2 = pd.read_excel('FINAL_topic_words_R2_review.xlsx')

In [ ]:
topic_final2.rename(columns={'word_lemma': 'topic_ng'}, inplace=True)


In [ ]:
topic_final2 = topic_final2[['topic','topic_ng']].drop_duplicates() # 933

In [ ]:
topic_final2 = topic_final2[topic_final2['topic_ng'] != 'choice'] # remove choice 


### round 2

In [ ]:
review_v2_review_st = review_v2_review[['similar_word', 'topic']]
review_v2_review_st.rename(columns={'similar_word': 'topic_ng'}, inplace=True)
review_v2_review_st = review_v2_review_st[['topic','topic_ng']].drop_duplicates() # 313

/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
len(review_v2_review_st)

313

In [ ]:

import re

def bigrams(s):
    return [i for i in s if re.search(r'\s', i) ]
  
def unigrams(s):
    return [i for i in s if not re.search(r'\s', i) ]


topic_words = list(set(review_v2_review_st['topic_ng'].tolist()))

word_bigrams_done = list(set(bigrams(topic_words)))
print(word_bigrams_done)
print(len(word_bigrams_done))
                       
word_unigrams_done = unigrams(topic_words)
print(word_unigrams_done)
print(len(word_unigrams_done))

['gas emission', 'source methane', 'emergency contraception', 'taxable income', 'conspiracy commit', 'illegally enter', 'reverence life', 'growth slow', 'anxiety screening', 'request asylum', 'venous thromboembolism', 'foreign national', 'arm dealer', 'death penalty', 'delta variant', 'blood thinner', 'certify presidential', 'brittany fonteno', 'coral reef', 'supply shortage', 'fda approve', 'deficit reduction', 'bodily autonomy', 'enroll medicare', 'arm trafficking', 'accusation sexual', 'dismiss illegitimate', 'knee surgery', 'unplanned pregnancy', 'break in', 'abbott bus', 'legal pathway', 'corona 19', 'financial institution', 'sleep habit', 'equity diversity', 'racial disparity', 'aggravate battery', 'american academy', 'attempt overturn', 'charge aggravated', 'birth defect', 'cost effective', 'gang relate', 'cognitive impairment', 'arrest allegedly', 'southern border', 'wildfire smoke', 'stock index', 'hearing aid', 'consumer spending', 'border mexico', 'primary caregiver', 'stock

In [ ]:

import numpy as np

df['topic_ng1'] = df['article_lemma_num'].astype(str).apply(lambda x: set.intersection(set(x.split(' ')), word_unigrams_done))
df['topic_ng2'] = df['article_lemma_num'].astype(str).apply(lambda x: [i for i in word_bigrams_done if i in x])


In [ ]:
df.sample(3)

,source,website,article_url,article_title,article_author,date_published,ner_places,article_lemma_num,article_final,translation_YN,id,ner_orgs,topic_ng1,topic_ng2
30481,Polk County Standard Journal,http://www.northwestgeorgianews.com/polkfishwrap/,https://www.northwestgeorgianews.com/fieldleve...,"Bret Bielema faces old team, Illinois visits W...",Field Level Media,2022-09-26,"{'Italy (GPE)', 'Canada (GPE)', 'Republic of I...",country united states america virgin islands u...,Country United States of America US Virgin Isl...,0,30482,"{'Grand Duchy of Macao (ORG)', 'Republic of Bo...",{},[]
1796,Douglas County Sentinel,http://www.douglascountysentinel.com/,https://www.douglascountysentinel.com/lifestyl...,AI Could Improve Diagnostic Accuracy of Breast...,Physician’s Briefing Staff,2022-10-07,"{'New York City (GPE)', 'Ph.D. (GPE)', 'Poland...",friday oct 7 2022 healthday news deep learning...,"FRIDAY, Oct. 7, 2022 (HealthDay News) -- A dee...",0,1797,"{'Science Translational Medicine (ORG)', 'BI-R...","{cancer, mri}",[cancer diagnosis]
125048,WAGA TV 5 (Fox),http://www.myfoxatlanta.com/,https://www.fox5atlanta.com/news/shaw-universi...,"Shaw University students stopped, searched in ...",FOX 5 Atlanta Digital Team,2022-10-11,"{'Atlanta (GPE)', 'Raleigh (GPE)', 'North Caro...",article exterior shaw university locate raleig...,"article Exterior of Shaw University, located i...",0,125049,"{'Shaw University (ORG)', 'the Center for Fina...",{},[]


In [ ]:

text_df_uni = df[(df['topic_ng1'].str.len()) > 0]
text_df_bi = df[(df['topic_ng2'].str.len()) > 0]

text_df_uni = text_df_uni[['id', 'source', 'article_final', 'topic_ng1']]
text_df_bi = text_df_bi[['id', 'source','article_final', 'topic_ng2']]

text_df_uni_df = text_df_uni.explode('topic_ng1')
text_df_bi_df = text_df_bi.explode('topic_ng2')

text_df_uni_df = text_df_uni_df.rename(columns={'topic_ng1': 'topic_ng'})
text_df_bi_df = text_df_bi_df.rename(columns={'topic_ng2': 'topic_ng'})

text_df_ng_all = pd.concat([text_df_uni_df, text_df_bi_df])



In [ ]:
text_df_ng_all = pd.merge(text_df_ng_all, review_v2_review_st, how='left', on = 'topic_ng')

In [ ]:
text_df_ng_all # three words missing

,id,source,article_final,topic_ng,topic
0,173704,285 South,The GOP effort to court Hispanic voters: Repub...,shooting,Guns
1,173702,285 South,The Guatemalan population in Georgia has almos...,migrate,Immigration / border security
2,173703,285 South,It\u2019s a busy month - lots of fundraisers a...,business,Economy / Jobs
3,173705,285 South,"Sara Naqvi, publisher of the blog The Pakistan...",business,Economy / Jobs
4,173705,285 South,"Sara Naqvi, publisher of the blog The Pakistan...",immigrate,Immigration / border security
...,...,...,...,...,...
102757,3813,WQXI AM 790,"Mi, 3 consecutive Giant Step… Return to the RO...",fed raise,Economy / Jobs
102758,3813,WQXI AM 790,"Mi, 3 consecutive Giant Step… Return to the RO...",raise interest,Economy / Jobs
102759,3814,WQXI AM 790,"2022 Fiscal year, more than 2 million immigran...",migrant cross,Immigration / border security
102760,3815,WQXI AM 790,The Fed's Economic Analyst is a 0.75 point inc...,fed raise,Economy / Jobs


In [ ]:
temp_count = text_df_ng_all.groupby(['topic', 'topic_ng']).count().reset_index()

In [ ]:
temp_count

,topic,topic_ng,id,source,article_final
0,Abortion,access reproductive,41,41,41
1,Abortion,bodily autonomy,35,35,35
2,Abortion,brittany fonteno,10,10,10
3,Abortion,fetal,243,243,243
4,Abortion,life unborn,24,24,24
...,...,...,...,...,...
306,Racial inequality,practice disproportionately,30,30,30
307,Racial inequality,prejudicial,31,31,31
308,Racial inequality,protest racial,22,22,22
309,Racial inequality,racial disparity,60,60,60


In [ ]:
# merge_uni_m10 = text_df_ng_all[~text_df_ng_all['topic_ng'].isin(temp_count_l10_topic)]
# merge_uni_m10_group = merge_uni_m10.groupby('topic_ng').apply(lambda x: x.sample(10, replace=False)).reset_index(drop=True)

# merge_uni_l10 = text_df_ng_all[text_df_ng_all['topic_ng'].isin(temp_count_l10_topic)]
# merge_uni_l10_group = merge_uni_l10.groupby('topic_ng').apply(lambda x: x.sample(10, replace=True)).reset_index(drop=True)

# merge_uni_both = pd.concat([merge_uni_m10_group, merge_uni_l10_group])

merge_uni_both = text_df_ng_all.groupby(['topic', 'topic_ng']).apply(lambda x: x.sample(10, replace=True)).reset_index(drop=True)


In [ ]:
merge_uni_both

,id,source,article_final,topic_ng,topic
0,141601,Decaturish,Decaturish and Tucker Observer sent candidate ...,access reproductive,Abortion
1,182746,WRBL News 3 (CBS),(NerdWallet) – Would you be able to quickly ob...,access reproductive,Abortion
2,180022,WJBF TV 6 (ABC),(NerdWallet) – Would you be able to quickly ob...,access reproductive,Abortion
3,48707,WGCL TV 46 (CBS),WASHINGTON (AP) — With few options for strengt...,access reproductive,Abortion
4,68691,WSB AM 750,As abortion remains a key issue in the upcomin...,access reproductive,Abortion
...,...,...,...,...,...
3105,19927,Atlanta Black Star,The white woman who made headlines in 2020 for...,racialize,Racial inequality
3106,186815,WSB TV 2 (ABC),"A single pair of Levi’s jeans, dating to the 1...",racialize,Racial inequality
3107,67742,WRFC AM 960,"A single pair of Levi’s jeans, dating to the 1...",racialize,Racial inequality
3108,67616,WRFC AM 960,Tulsi Gabbard says she is leaving the Democrat...,racialize,Racial inequality


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
by_sentence = merge_uni_both['article_final'].apply(lambda x : sent_tokenize(x)).apply(pd.Series,1).stack()
by_sentence_df = pd.DataFrame(by_sentence)
by_sentence_df.rename(columns={0:'sentence_each'}, inplace=True)

by_sentence_df['sentence_turn'] = by_sentence_df.index.get_level_values(1)
by_sentence_df.index = by_sentence_df.index.get_level_values(0)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
merge_all_join = pd.concat([merge_uni_both, by_sentence_df], axis=1)
merge_all_join['sentence_each_lemma'] = merge_all_join['sentence_each'].str.lower().astype(str).apply(get_lemmas_num)



In [ ]:
merge_all_join['topic_words_YN'] = merge_all_join.apply(lambda x: x.topic_ng in x.sentence_each_lemma, axis=1)
merge_all_join['topic_words_YN_lead'] = merge_all_join.groupby(['topic', 'topic_ng'])['topic_words_YN'].shift(-1)
merge_all_join.reset_index(drop=True, inplace=True)



In [ ]:
merge_all_join["row_id"] = merge_all_join.index + 1


In [ ]:
merge_sentence_select_in = merge_all_join[ (merge_all_join['topic_words_YN']== True)]


In [ ]:
merge_sentence_only = merge_sentence_select_in.drop(columns=['article_final'])
merge_sentence_only['str_count'] = merge_sentence_only.sentence_each.str.count(" ") # remove string count less than 3

In [ ]:
merge_sentence_only = merge_sentence_only[merge_sentence_only['str_count'] >= 3]

In [ ]:
merge_sentence_only_10 = merge_sentence_only.groupby(['topic', 'topic_ng']).apply(lambda x: x.sample(10, replace=True)).reset_index(drop=True)


In [ ]:
test = merge_sentence_only_10.groupby(['topic', 'topic_ng']).count().reset_index()
test

,topic,topic_ng,id,source,sentence_each,sentence_turn,sentence_each_lemma,topic_words_YN,topic_words_YN_lead,row_id,str_count
0,Abortion,access reproductive,10,10,10,10,10,10,10,10,10
1,Abortion,bodily autonomy,10,10,10,10,10,10,10,10,10
2,Abortion,brittany fonteno,10,10,10,10,10,10,10,10,10
3,Abortion,fetal,10,10,10,10,10,10,10,10,10
4,Abortion,life unborn,10,10,10,10,10,10,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...
306,Racial inequality,practice disproportionately,10,10,10,10,10,10,10,10,10
307,Racial inequality,prejudicial,10,10,10,10,10,10,10,10,10
308,Racial inequality,protest racial,10,10,10,10,10,10,10,10,10
309,Racial inequality,racial disparity,10,10,10,10,10,10,10,10,10


In [ ]:
merge_sentence_only_10.sample(3)

,id,source,topic_ng,topic,sentence_each,sentence_turn,sentence_each_lemma,topic_words_YN,topic_words_YN_lead,row_id,str_count
1305,417,La Visión,price gasoline,Economy / Jobs,The prices of gasoline in the Metropolitan Are...,0,price gasoline metropolitan area atlanta shoot...,True,False,49776,29
1882,141132,Dalton Daily Citizen,cardiomyopathy,Health care,His mother said jailers refused to take him to...,14,mother say jailer refuse hospital medical care...,True,False,70731,19
636,26801,Lanier County News,armed dangerous,Crime,Jashun Wilkins is considered armed and dangerous.,14,jashun wilkin consider armed dangerous,True,False,25765,6


In [ ]:
merge_sentence_only_10_st = merge_sentence_only_10[['id','topic_ng','topic','sentence_each']]

In [ ]:
# # remove choice

# merge_sentence_only_10_st = merge_sentence_only_10_st[merge_sentence_only_10_st['topic_ng'] != 'choice']

In [ ]:
len(merge_sentence_only_10_st)

3110

In [ ]:
merge_sentence_only_10_st = merge_sentence_only_10_st.sample(len(merge_sentence_only_10_st))
merge_sentence_only_10_st.to_excel('Georgia_topic_words_validation_sentence10_v2.xlsx')